In [1]:
import numpy as np
import random

### Task 1 : Solve a 8-Queens problem 

#### Function to create a Random Chromosome (Solution)

In [2]:
#The function creates a random individual/chromosome representing a possible solution
def create_chromosome(N):
    return random.sample(range(N),N)

#### Function to calculate the Fitness score of a chromosome

In [3]:
#This function calculates the fitness of an individual/chromosome
#The functions calclutes the number of non-clashes between the queens
#higher the fitness score better the fitness of the chromosome.
#The highest fitness score for an 8-Queen solution  is 28 (when the chromosome has 0 clashes) and 
#For n queens max fitness score will be (n-1)/2
def calculate_fitness(chromosome):
    # Calculate the fitness of a chromosome
    clashes = 0
    q_count = len(chromosome)
    max_score = q_count*(q_count-1)/2
    #print("max_count = {}".format(max_score) )
    for i in range(q_count):
        for j in range(i + 1, q_count):
            if chromosome[i] == chromosome[j] or abs(chromosome[i] - chromosome[j]) == j - i:
                clashes += 1
    return max_score - clashes

#### Function to select Parents for offsprings

In [4]:
#This function selects the parents for the new offsprings from the population
#Parents are selected based on their fitness score,those with higher fitness score have higher probability of being selected
#1. Find the Total fitness score of all the chromosomes in population
#2. Calculate the probablity of each individual chromosome, as a percentage of the total fitness score
#3. Randomly pick two chromosomes as parents (weighted by their probability)
def select_parents(population):
    total_population_fitness = sum(calculate_fitness(chromosome) for chromosome in population)
    probabilities = [calculate_fitness(chromosome) / total_population_fitness for chromosome in population]
    return random.choices(population, probabilities, k=2)
    

#### Single point crossover function

In [5]:
#This function crosses over or combines the two parents to create an offspring
#This is a simple implementation of cross over (called single point converter), 
#where a single crossover point is chosen and genes are exchanged between parents beyond the crossover point
def single_point_crossover(parent1, parent2):
    if(len(parent1)!=len(parent2)):
        print("Parents not of same length")
        exit
    crossover_point = random.randint(1, len(parent1) - 1)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    return offspring1, offspring2

#### Function to mutate chromosome

In [6]:
#This function mutates the chromosome, if the mutation_rate is greater than a random value,
#The chromosome is mutated by switching randomly selected genes,   
def mutate_chromosome(chromosome,mutation_rate):
    if random.random() < mutation_rate:
        index1, index2 = random.sample(range(len(chromosome)), 2)
        chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]
    return chromosome

### Genetic Algorithm 

In [7]:
#This function implements the Genetic Algorithm for N-Queens, 
#although it can be used for higher or lower number of Queens too, 
#currently it is being used for 8 or lower Queens
def genetic_algorithm(N,POP_SIZE,MAX_GENERATIONS,MUTATION_RATE):
    population = [create_chromosome(N) for _ in range(POP_SIZE)]
    generation = 0

    while generation < MAX_GENERATIONS:
        new_population = []

        while len(new_population) < POP_SIZE:
            parent1, parent2 = select_parents(population)
            child1, child2 = single_point_crossover(parent1, parent2)
            child1 = mutate_chromosome(child1,MUTATION_RATE)
            child2 = mutate_chromosome(child2,MUTATION_RATE)
            new_population.extend([child1, child2])

        population = new_population
        generation += 1

        # Check for a solution, for  8 Queens the solution will have a fitness score of 28 (no clashes) 
        for chromosome in population:
            if calculate_fitness(chromosome) == N * (N - 1) / 2:
                return chromosome

    # No solution found
    return None



### Run the Genetic Algorithm for 8-Queens

In [23]:
#Run the Genetic algorithm for 8-Queens with 
#1.Random Population
#2.Single point crossover and
#3.Random mutation < 0.1
#4.Population Size = 100
#5.Running it upto 100 times 
from datetime import datetime
now = datetime.now()
flag = False
for i in range (100):
    solution = genetic_algorithm(8,100,10,0.1)
    if solution:
        print("Solution found: {} , iteration = {}".format(solution,i))
        later = datetime.now()
        print("Time Taken = {}".format(later-now))
        flag = True
        break
    else:
        continue

if flag == False:
    print("No Solution found")

Solution found: [0, 6, 3, 5, 7, 1, 4, 2] , iteration = 19
Time Taken = 0:00:24.333782


### Task 2 : Solve N- Queen problem ,adding elitism to population creation
###### For task 2 we are using elitism , where fittest chromosomes from the original population are retained in the new population and also free to be chosen as parents.

#### Function to generate elite population : 

In [9]:
#This function Generates population with elitism 
#Elitism is introduced by retaining top 10% chromosomes from population in the next population)
def create_elite_population(population,MUTATION_RATE):
    
    #Sort the population on fitness  
    sorted_population = sorted(population, key=lambda x: calculate_fitness(x),reverse=True)
    #print(sorted_population)
    #Add 10% of previous population to the new population
    new_population = sorted_population[:int(len(sorted_population) * 0.1)]
    
    #Now create Rest of the new population by selecting, crossover and mutation
    while len(new_population) < len(population):
        parent1, parent2 = select_parents(population)
        child1, child2 = single_point_crossover(parent1, parent2)
        child1 = mutate_chromosome(child1,MUTATION_RATE)
        child2 = mutate_chromosome(child2,MUTATION_RATE)
        new_population.extend([child1,child2])
    
    return new_population

In [22]:
 def generate_algorithm_N(N,POP_SIZE,MAX_GENERATIONS,MUTATION_RATE):
    population = [create_chromosome(N) for _ in range(POP_SIZE)]
    
    for generation in range(MAX_GENERATIONS):
        #print(f"Generation {generation + 1}")
        best_individual = sorted(population, key=lambda x: calculate_fitness(x),reverse=True)[0]
        #print(f"Best Individual's Fitness: {calculate_fitness(best_individual)}")
                
        if calculate_fitness(best_individual) == N*(N-1)/2:
            print("Solution found!")
            return best_individual
            
        population = create_elite_population(population,MUTATION_RATE)
    
    if calculate_fitness(best_individual) != N*(N-1)/2:
        print("Solution not found.")

##### Run for N=4

In [11]:
from datetime import datetime
now = datetime.now()
solution = generate_algorithm_N(4,100,100,0.1)
later = datetime.now()
if solution:
    print("Solution found: {}".format(solution))
    print("Time Taken = {}".format(later-now))
else:
    print("No Solution found")

Solution found!
Solution found: [2, 0, 3, 1]
Time Taken = 0:00:00.002998


##### Run for N=8

In [12]:
now = datetime.now()
solution = generate_algorithm_N(8,100,100,0.1)
later = datetime.now()
if solution:
    print("Solution found: {}".format(solution))
    print("Time Taken = {}".format(later-now))
else:
    print("No Solution found")

Solution found!
Solution found: [3, 0, 4, 7, 1, 6, 2, 5]
Time Taken = 0:00:03.617190


##### Run for N=12

In [17]:
now = datetime.now()
solution = generate_algorithm_N(12,100,100,0.1)
later = datetime.now()
if solution:
    print("Solution found: {}".format(solution))
    print("Time Taken = {}".format(later-now))
else:
    print("No Solution found")

Solution found!
Solution found: [2, 7, 5, 11, 1, 10, 6, 3, 9, 4, 8, 0]
Time Taken = 0:00:13.349487


##### Run for N=16

In [20]:
now = datetime.now()
solution = generate_algorithm_N(16,100,1000,0.1)
later = datetime.now()
if solution:
    print("Solution found: {}".format(solution))
    print("Time Taken = {}".format(later-now))
else:
    print("No Solution found")

Solution found!
Solution found: [6, 14, 2, 7, 1, 13, 15, 9, 11, 5, 3, 12, 0, 4, 8, 10]
Time Taken = 0:00:40.502837


##### Run for N=20

In [25]:
now = datetime.now()
solution = generate_algorithm_N(20,100,1000,0.1)
later = datetime.now()
if solution:
    print("Solution found: {}".format(solution))
    print("Time Taken = {}".format(later-now))
else:
    print("No Solution found")

Solution found!
Solution found: [11, 7, 0, 3, 6, 15, 10, 19, 5, 12, 18, 4, 17, 9, 13, 16, 2, 8, 1, 14]
Time Taken = 0:01:27.082790
